In [ ]:
import sys
import torch
import time
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLabel, QPushButton, QVBoxLayout,
    QHBoxLayout, QFileDialog, QTextEdit, QGridLayout
)
from PyQt5.QtGui import QFont
from PyQt5.QtCore import Qt

class RefinedDroneDashboard(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Intelligent Drone Surveillance Dashboard")
        self.setGeometry(100, 100, 1000, 600)
        self.setStyleSheet("background-color: #111827; color: white;")
        self.model, self.processor = self.load_model()
        self.setup_ui()

    def load_model(self):
        model_name = "Qwen/Qwen2-VL-2B-Instruct"
        model = Qwen2VLForConditionalGeneration.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            attn_implementation="eager",
            device_map={"": "cuda"}
        )
        processor = AutoProcessor.from_pretrained(model_name, use_fast=True)
        return model, processor

    def setup_ui(self):
        main_layout = QHBoxLayout()

        # Left Panel
        left_layout = QVBoxLayout()
        title_label = QLabel("🛡️ Intelligent Drone Surveillance")
        title_label.setFont(QFont("Arial", 16, QFont.Bold))

        self.upload_label = QLabel("No file selected")
        self.upload_button = QPushButton("Browse Image/Video...")
        self.upload_button.clicked.connect(self.browse_file)
        self.upload_button.setStyleSheet("background-color: #1f2937; color: white; padding: 8px;")

        upload_box = QVBoxLayout()
        upload_box.addWidget(self.upload_label)
        upload_box.addWidget(self.upload_button)

        upload_frame = QWidget()
        upload_frame.setLayout(upload_box)
        upload_frame.setStyleSheet("border: 1px solid #374151; padding: 15px; border-radius: 10px;")


        report_label = QLabel("📄 Detailed Report")
        self.report_text = QTextEdit()
        self.report_text.setPlaceholderText("Full report...")
        self.report_text.setStyleSheet("background-color: #1f2937;")

        download_btn = QPushButton("Download PDF")
        download_btn.setStyleSheet("background-color: #3b82f6; color: white; padding: 10px;")

        left_layout.addWidget(title_label)
        left_layout.addWidget(upload_frame)
        left_layout.addSpacing(20)
        left_layout.addWidget(report_label)
        left_layout.addWidget(self.report_text)
        left_layout.addWidget(download_btn)

        # Right Panel
        right_layout = QVBoxLayout()
        dashboard_title = QLabel("📊 Dashboard")
        dashboard_title.setFont(QFont("Arial", 16, QFont.Bold))

        grid = QGridLayout()
        grid.setSpacing(20)

        metrics = [
            ("🛸", "Hours Flown", "12.3"),
            ("🛢️", "Fuel Capacity", "100.0"),
            ("⛽", "Remaining Fuel", "65.7"),
            ("🗺️", "Explored Area", "45.8"),
            ("📸", "Captured Images", "234"),
        ]

        for i, (icon, label, value) in enumerate(metrics):
            icon_label = QLabel(icon)
            icon_label.setFont(QFont("Arial", 36))

            metric_label = QLabel(label)
            metric_label.setFont(QFont("Arial", 14, QFont.Bold))

            value_label = QLabel(value)
            value_label.setFont(QFont("Arial", 20, QFont.Bold))
            value_label.setStyleSheet("color: #60a5fa;")

            row_layout = QVBoxLayout()
            row_layout.setAlignment(Qt.AlignCenter)
            row_layout.addWidget(icon_label)
            row_layout.addWidget(metric_label)
            row_layout.addWidget(value_label)

            cell = QWidget()
            cell.setLayout(row_layout)
            cell.setStyleSheet("background-color: #1f2937; padding: 20px; border-radius: 10px;")
            grid.addWidget(cell, i // 2, i % 2)

        dashboard_widget = QWidget()
        dashboard_widget.setLayout(grid)

        right_layout.addWidget(dashboard_title)
        right_layout.addWidget(dashboard_widget)

        main_layout.addLayout(left_layout, 2)
        main_layout.addLayout(right_layout, 2)

        self.setLayout(main_layout)

    def browse_file(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Select Image or Video", "", "Media Files (*.jpg *.jpeg *.png *.mp4 *.mov *.avi)")
        if file_path:
            self.upload_label.setText(file_path.split("/")[-1])
            self.run_model(file_path)

    def run_model(self, file_path):
        try:
            is_video = file_path.lower().endswith((".mp4", ".mov", ".avi"))
            input_type = "video" if is_video else "image"

            messages = [{
                "role": "user",
                "content": [
                    {"type": input_type, input_type: file_path},
                    {"type": "text", "text": "Explain this {} in a detailed manner.".format(input_type)}
                ]
            }]

            st = time.time()
            processor = self.processor
            model = self.model

            text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            image_inputs, video_inputs = process_vision_info(messages)

            inputs = processor(
                text=[text],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt"
            ).to("cuda")

            generated_ids = model.generate(**inputs, max_new_tokens=512)
            trimmed = [o[len(i):] for i, o in zip(inputs.input_ids, generated_ids)]
            output_text = processor.batch_decode(trimmed, skip_special_tokens=True)
            self.report_text.setPlainText("Detailed report based on {}:\n".format(input_type) + output_text[0])
            print("Time taken to generate:", time.time() - st)

            torch.cuda.empty_cache()

        except Exception as e:
            self.report_text.setPlainText("Error: " + str(e))

if __name__ == "__main__":
    app = QApplication(sys.argv)
    dashboard = RefinedDroneDashboard()
    dashboard.show()
    sys.exit(app.exec_())


C:\Users\kmano\miniconda3\envs\lstr\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.51s/it]


Time taken to generate: 4.343367099761963


In [ ]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [ ]:
import sys
import torch
import time
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLabel, QPushButton, QVBoxLayout,
    QHBoxLayout, QFileDialog, QTextEdit, QGridLayout, QMessageBox
)
from PyQt5.QtGui import QFont
from PyQt5.QtCore import Qt

class RefinedDroneDashboard(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Intelligent Drone Surveillance Dashboard")
        self.setGeometry(100, 100, 1000, 600)
        self.setStyleSheet("background-color: #111827; color: white;")
        
        self.model = None
        self.processor = None
        self.file_path = None
        
        self.setup_ui()

    def setup_ui(self):
        main_layout = QHBoxLayout()

        # Left Layout
        left_layout = QVBoxLayout()
        title_label = QLabel("🛡️ Intelligent Drone Surveillance")
        title_label.setFont(QFont("Arial", 16, QFont.Bold))

        # Load model button
        self.load_model_button = QPushButton("Load Qwen2-VL Model")
        self.load_model_button.setStyleSheet("background-color: #10b981; color: white; padding: 10px;")
        self.load_model_button.clicked.connect(self.load_model)

        # Upload Section
        self.upload_label = QLabel("No file selected")
        self.upload_button = QPushButton("Browse...")
        self.upload_button.setEnabled(False)
        self.upload_button.setStyleSheet("background-color: #1f2937; color: white; padding: 8px;")
        self.upload_button.clicked.connect(self.browse_file)

        upload_box = QVBoxLayout()
        upload_box.addWidget(self.upload_label)
        upload_box.addWidget(self.upload_button)

        upload_frame = QWidget()
        upload_frame.setLayout(upload_box)
        upload_frame.setStyleSheet("border: 1px solid #374151; padding: 15px; border-radius: 10px;")

        # Summary + Report
        summary_label = QLabel("📝 Generated Summary")
        self.summary_text = QTextEdit()
        self.summary_text.setReadOnly(True)
        self.summary_text.setStyleSheet("background-color: #1f2937;")

        report_label = QLabel("📄 Detailed Report")
        self.report_text = QTextEdit()
        self.report_text.setReadOnly(True)
        self.report_text.setStyleSheet("background-color: #1f2937;")

        # Generate button
        self.generate_button = QPushButton("Generate Detailed Report")
        self.generate_button.setEnabled(False)
        self.generate_button.setStyleSheet("background-color: #3b82f6; color: white; padding: 10px;")
        self.generate_button.clicked.connect(self.run_model)

        # Assemble left layout
        left_layout.addWidget(title_label)
        left_layout.addWidget(self.load_model_button)
        left_layout.addSpacing(20)
        left_layout.addWidget(upload_frame)
        left_layout.addSpacing(20)
        left_layout.addWidget(summary_label)
        left_layout.addWidget(self.summary_text)
        left_layout.addWidget(report_label)
        left_layout.addWidget(self.report_text)
        left_layout.addWidget(self.generate_button)

        # Right Layout (Dashboard)
        right_layout = QVBoxLayout()
        dashboard_title = QLabel("📊 Dashboard")
        dashboard_title.setFont(QFont("Arial", 16, QFont.Bold))

        grid = QGridLayout()
        grid.setSpacing(20)

        metrics = [
            ("🛸", "Hours Flown", "12.3"),
            ("🛢️", "Fuel Capacity", "100.0"),
            ("⛽", "Remaining Fuel", "65.7"),
            ("🗺️", "Explored Area", "45.8"),
            ("📸", "Captured Images", "234"),
        ]

        for i, (icon, label, value) in enumerate(metrics):
            icon_label = QLabel(icon)
            icon_label.setFont(QFont("Arial", 36))

            metric_label = QLabel(label)
            metric_label.setFont(QFont("Arial", 14, QFont.Bold))

            value_label = QLabel(value)
            value_label.setFont(QFont("Arial", 20, QFont.Bold))
            value_label.setStyleSheet("color: #60a5fa;")

            row_layout = QVBoxLayout()
            row_layout.setAlignment(Qt.AlignCenter)
            row_layout.addWidget(icon_label)
            row_layout.addWidget(metric_label)
            row_layout.addWidget(value_label)

            cell = QWidget()
            cell.setLayout(row_layout)
            cell.setStyleSheet("background-color: #1f2937; padding: 20px; border-radius: 10px;")
            grid.addWidget(cell, i // 2, i % 2)

        dashboard_widget = QWidget()
        dashboard_widget.setLayout(grid)

        right_layout.addWidget(dashboard_title)
        right_layout.addWidget(dashboard_widget)

        main_layout.addLayout(left_layout, 2)
        main_layout.addLayout(right_layout, 2)

        self.setLayout(main_layout)

    def load_model(self):
        try:
            self.load_model_button.setText("Loading Model...")
            QApplication.processEvents()  # update UI

            model_name = "Qwen/Qwen2-VL-2B-Instruct"
            self.model = Qwen2VLForConditionalGeneration.from_pretrained(
                model_name,
                torch_dtype=torch.bfloat16,
                attn_implementation="eager",
                device_map={"": "cuda"}
            )
            self.processor = AutoProcessor.from_pretrained(model_name, use_fast=True)

            self.upload_button.setEnabled(True)
            self.generate_button.setEnabled(True)
            self.load_model_button.setText("Model Loaded ✅")

        except Exception as e:
            self.load_model_button.setText("Load Qwen2-VL Model")
            QMessageBox.critical(self, "Error", f"Failed to load model:\n{e}")

    def browse_file(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Select File")
        if file_path:
            self.upload_label.setText(file_path.split("/")[-1])
            self.file_path = file_path

    def run_model(self):
        if not self.model or not self.processor:
            QMessageBox.warning(self, "Model Not Loaded", "Please load the model first.")
            return

        if not self.file_path:
            QMessageBox.warning(self, "No File", "Please upload an image or video file first.")
            return

        try:
            messages = [{
                "role": "user",
                "content": [
                    {"type": "image", "image": self.file_path},
                    {"type": "text", "text": "explain this image in detail."}
                ]
            }]

            start = time.time()
            text = self.processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            image_inputs, video_inputs = process_vision_info(messages)

            inputs = self.processor(
                text=[text],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt"
            ).to("cuda")

            output_ids = self.model.generate(**inputs, max_new_tokens=512)
            trimmed_ids = [o[len(i):] for i, o in zip(inputs.input_ids, output_ids)]
            output_text = self.processor.batch_decode(trimmed_ids, skip_special_tokens=True)

            summary = output_text[:]
            self.summary_text.setPlainText(summary)
            self.report_text.setPlainText("📄 Detailed Report\n\n" + summary)
            print("Inference time:", time.time() - start)

            torch.cuda.empty_cache()

        except Exception as e:
            self.summary_text.setPlainText("Error: " + str(e))
            self.report_text.clear()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    dashboard = RefinedDroneDashboard()
    dashboard.show()
    sys.exit(app.exec_())


In [ ]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()